# 五冰湖ICESat-2，子数据下载
采用矩形获取选区交点
5湖GEE basemap下载
采用Arcgis截图下载

In [ ]:
import icepyx as ipx
import geopandas as gpd
import os
from pyproj import Transformer

# 定义坐标系
pcs_3857 = 'EPSG:3857'  # （单位：米）
pcs_4326 = 'EPSG:4326'  # （单位：经纬度）

# 读取湖泊边界数据并转换为 pcs_3857
dir_path = r"C:\Users\Administrator\Desktop\test"
shp_files = gpd.read_file(os.path.join(dir_path, 'lakes_boundaries', '5lake_domain_merge.shp'))
shp_files = shp_files.to_crs(pcs_3857)  # 转换为投影坐标系（单位：米）

# 创建坐标转换器（从 3857 到 4326）
transformer = Transformer.from_crs(pcs_3857, pcs_4326, always_xy=True)

for idx, row in shp_files.iterrows():
    try:
        name = row.get('name', f"lake_{idx}")  # 获取湖泊名称
        
        buffered_geom = row.geometry.buffer(100)  # 缓冲100米
        
        minx, miny, maxx, maxy = buffered_geom.bounds
        minx_lon, miny_lat = transformer.transform(minx, miny)
        maxx_lon, maxy_lat = transformer.transform(maxx, maxy)
        
        # 定义四个角点（顺时针顺序）
        top_left = (minx_lon, maxy_lat)
        top_right = (maxx_lon, maxy_lat)
        bottom_right = (maxx_lon, miny_lat)
        bottom_left = (minx_lon, miny_lat)
        spatial_extent = [top_left, top_right, bottom_right, bottom_left, top_left]
        
        print(f"处理湖泊: {name}")
        print(f"空间范围: {spatial_extent}")

        # ICESat-2查询参数
        beam_list = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
        var_list = ['h_ph', 'lat_ph', 'lon_ph', 'quality_ph', 'signal_conf_ph']
        date_range = {"start_date": '2015-01-01', "end_date": '2025-04-30'}
        
        # 创建查询
        region_a = ipx.Query(
            'ATL03', 
            spatial_extent, 
            date_range, 
            version='006'
        )
        region_a.avail_granules(ids=True)
        
        # 设置下载参数（修正此处语法错误）
        region_a.order_vars.append(var_list=var_list)  # 仅添加变量列表
        region_a.subsetparams(Coverage=region_a.order_vars.wanted)
        
        # 下载数据（保存到以湖泊名称命名的子目录）
        download_path = os.path.join(dir_path, "ICESat-2_h5_files", f"{name}")
        os.makedirs(download_path, exist_ok=True)  # 创建目录（如果不存在）
        region_a.download_granules(download_path)
        
        print(f"成功下载湖泊 {name} 的数据到 {download_path}")
    
    except Exception as e:
        print(f"处理湖泊 {name} 时出错: {str(e)}")
        continue  # 跳过当前错误，继续下一个

# 获取冰湖与ICESat-2存在交集
合并所有h5文件，并转化为pandas dataframe

In [ ]:
import h5py
import pandas as pd
import os
import timescale
import numpy as np
from shapely.geometry import Point

def make_dir(path):
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        os.makedirs(path)
        print(path + ' 创建成功')
        return path
    return path

def convert_to_dataframe(group, prefix=''):
    """改进版：确保每个数据集列名唯一且匹配"""
    dataframes = {}
    for key in group.keys():
        item = group[key]
        if isinstance(item, h5py.Dataset):
            data = item[:]
            if data.ndim == 1:
                dataframes[key] = pd.Series(data)
            else:
                # 多维数组：拆分为多列，列名添加后缀
                df = pd.DataFrame(data)
                dataframes.update({f"{key}_{i}": df.iloc[:, i] for i in range(df.shape[1])})
        elif isinstance(item, h5py.Group):
            dataframes.update(convert_to_dataframe(item, f"{prefix}/{key}" if prefix else key))
    return dataframes

def convert_delta_time(delta_time, gps_epoch=1198800018.0):
    delta_time = np.atleast_1d(delta_time)
    gps_seconds = gps_epoch + delta_time
    time_leaps = timescale.time.count_leap_seconds(gps_seconds)
    time_julian = 2400000.5 + timescale.time.convert_delta_time(
        gps_seconds - time_leaps, epoch1=(1980,1,6,0,0,0),
        epoch2=(1858,11,17,0,0,0), scale=1.0/86400.0)
    Y, M, D, h, m, s = timescale.time.convert_julian(time_julian, format='tuple')
    Y, M, D = int(Y[0]), int(M[0]), int(D[0])
    return f'{Y}-{M:02d}-{D:02d}'

def process_h5_file(filename):
    """处理单个HDF5文件并返回DataFrame"""
    beam_list = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
    with h5py.File(filename, 'r') as h5file:
        # 初始化结果DataFrame
        result_df = pd.DataFrame()
        
        # 处理orbit_info数据
        if 'orbit_info' in h5file:
            orbit_data = convert_to_dataframe(h5file['orbit_info'])
            orbit_data['crossing_time'] = convert_delta_time(orbit_data['crossing_time'])
            orbit_data['sc_orient_time'] = convert_delta_time(orbit_data['sc_orient_time'])
            orbit_df = pd.DataFrame({k: v for k, v in orbit_data.items() 
                                if not k.startswith('geolocation')})
            result_df = pd.concat([result_df, orbit_df], axis=1)
        
        # 处理每个beam的数据
        beam_dfs = []
        for beam in beam_list:
            if beam in h5file:
                beam_data = convert_to_dataframe(h5file[beam])
                beam_df = pd.DataFrame({k: v for k, v in beam_data.items()
                                    if not k.startswith('geolocation')})
                beam_df['beam'] = beam  # 添加beam标记
                beam_dfs.append(beam_df)
        
        # 合并所有beam数据
        if beam_dfs:
            all_beams_df = pd.concat(beam_dfs, axis=0, ignore_index=True)
            result_df = pd.concat([result_df, all_beams_df], axis=1)
        
        # 填充NaN值
        result_df = result_df.fillna(method='ffill')
        
        # 添加文件名列
        result_df['file_name'] = os.path.basename(filename)
        
        # 分类逻辑（更新列名引用）
        def classify_ph(row):
            signal_cols = [c for c in row.index if 'signal_conf_ph' in c]
            if not signal_cols:
                return 'Unknown'
            max_val = row[signal_cols].max()
            if max_val == 4: return 'High'
            elif max_val == 3: return 'Median'
            elif max_val == 2: return 'Low'
            elif max_val == 1: return 'Buffer'
            elif max_val == 0: return 'Noise'
            else: return 'Unknown'
        
        result_df['classification'] = result_df.apply(classify_ph, axis=1)
        
        return result_df
    
# 将这些点与湖泊矢量进行取交集，保留相交部分
def filter_points_by_polygon(df, polygon, lon_col='lon_ph', lat_col='lat_ph'):
    """
    使用多边形筛选数据点
    
    参数:
        df (pd.DataFrame): 包含经纬度的数据框
        polygon (shapely.Polygon): 用于筛选的多边形
        lon_col (str): 经度列名
        lat_col (str): 纬度列名
    
    返回:
        pd.DataFrame: 筛选后的数据框
    """
    # 创建GeoDataFrame
    geometry = [Point(lon, lat) for lon, lat in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    
    # 执行空间筛选
    mask = gdf.geometry.within(polygon)
    return gdf[mask].drop(columns=['geometry'])

# 主处理流程
dir_path = r"C:\Users\Administrator\Desktop\test"
folders = os.listdir(os.path.join(dir_path, 'ICESat-2_h5_files'))
h5_files = 'ICESat-2_h5_files'

# 加载湖泊边界
boundary_path = "5lake_domain_merge.shp"
lake_boundary = gpd.read_file(os.path.join(dir_path,'lakes_boundaries',boundary_path))

for lake_name in folders:
    print(f"Processing lake: {lake_name}")

    output_dir = make_dir(os.path.join(dir_path,'filter_ATL03'))
    filter_geom = lake_boundary[lake_boundary.name == lake_name].geometry.buffer(100).to_crs("EPSG:4326")

    if len(filter_geom) == 0:
        raise ValueError(f"未找到名为 {lake_name} 的湖泊边界数据")
    filter_polygon = filter_geom.iloc[0]

    # 处理所有HDF5文件
    all_files = [os.path.join(dir_path,h5_files,lake_name,f) for f in os.listdir(os.path.join(dir_path,h5_files,lake_name)) if f.endswith('.h5')]
    final_df = pd.DataFrame()

    for filename in all_files:
        try:
            print(f"Processing {filename}...")
            df = process_h5_file(filename)
            filtered_df = filter_points_by_polygon(df, filter_polygon)
            final_df = pd.concat([final_df, filtered_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            continue

    # 保存结果
    output_path = f"{output_dir}/{lake_name}_filtered.csv"
    final_df.to_csv(output_path, index=False)
    print(f"处理完成，共保留 {len(final_df)} 个点，结果已保存至 {output_path}")

# 获取冰湖的湖面高度(lake_level)，并修正对应的湖泊测深图
用于有湖泊测深验证，无验证、预测可以跳过该步骤

In [ ]:
import numpy as np
from scipy import stats
import rasterio
import geopandas as gpd
import pandas as pd

# 计算ICESat-2湖面高度
dir_path = r"C:\Users\Administrator\Desktop\test"
csv_dir = 'filter_ATL03'
bathymetry_dir = 'lakes_Bathymetry'

folders = os.listdir(os.path.join(dir_path, 'ICESat-2_h5_files'))

for name in folders:
    final_df = gpd.read_file(os.path.join(dir_path,csv_dir,f'{name}_filtered.csv'))
    
    # 确保h_ph列是数值类型
    final_df['h_ph'] = pd.to_numeric(final_df['h_ph'], errors='coerce')

    # 过滤数据
    filtered_df = final_df[~final_df['classification'].isin(['Unknown', 'Noise'])]
    elevations = filtered_df['h_ph'].values

    # 检查并移除NaN值
    elevations = elevations[~np.isnan(elevations)]

    # (1) 计算众数（使用np.unique替代scipy.stats.mode）
    unique_values, counts = np.unique(elevations, return_counts=True)
    mode = unique_values[np.argmax(counts)]
    print(f"众数（湖面高度估计）: {mode:.2f} m")

    # (2) 计算KDE峰值
    kde = stats.gaussian_kde(elevations)
    x = np.linspace(elevations.min(), elevations.max(), 1000)
    y = kde(x)
    lake_height_kde = x[y.argmax()]
    print(f"KDE 峰值（湖面高度估计）: {lake_height_kde:.2f} m")

    # 计算平均湖面高度
    lake_level = (mode + lake_height_kde)/2
    print(f"湖面高度(KDE与众数均值): {lake_level:.2f} m")

    # 修正湖泊深度图
    DEM_path = os.path.join(dir_path, bathymetry_dir, f"LakeBathymetry_{name}.tif")

    # 读取DEM数据
    with rasterio.open(DEM_path) as src:
        dem = src.read(1)
        profile = src.profile
        
        # 获取nodata值，如果没有则设为np.nan
        nodata = src.nodatavals[0] if src.nodatavals[0] is not None else np.nan
        
        # 创建掩码，排除nodata值
        if np.isnan(nodata):
            valid_mask = ~np.isnan(dem)
        else:
            valid_mask = dem != nodata
        
        # 统计DEM有效值范围
        dem_valid = dem[valid_mask]
        if len(dem_valid) == 0:
            raise ValueError("DEM中不包含有效数据")
        
        dem_min = np.min(dem_valid)
        dem_max = np.max(dem_valid)
        print(f"\nDEM有效值范围: {dem_min:.2f} 到 {dem_max:.2f}")
        
        # 判断是高度图还是深度图
        if dem_min >= 1:  # 假设高度图通常高于湖面
            print("检测到高度图，转换为高度图...")
            # 计算湖面与DEM最高点的差距
            depth = dem_max - lake_level
            # 转换为深度图（0表示湖面，正值表示水深）
            corrected_dem = np.where(valid_mask, dem-depth, nodata)
            
        else:
            print("检测到深度图，转换为高度图...")
            # 转换为高度图（湖面为基准）
            corrected_dem = np.where(valid_mask, lake_level - dem, nodata)
        
        # 更新profile中的nodata值
        profile.update(nodata=nodata)
        
        # 保存修正后的DEM
        output_path = DEM_path.replace(".tif", f"_corrected.tif")
        with rasterio.open(output_path, 'w',**profile) as dst:
            dst.write(corrected_dem, 1)
        print(f"已保存修正后的DEM到: {output_path}")

# 下载basemap
可以选择其他方式获取basemap，论文中采用Planet底图 

In [ ]:
import os
import numpy as np
import geopandas as gpd
import mercantile
import requests
from io import BytesIO
from PIL import Image
import rasterio
from rasterio.transform import from_bounds
from rasterio.mask import mask
from pyproj import Transformer
import concurrent.futures
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# 定义坐标系
pcs_3857 = 'EPSG:3857'  # Web墨卡托
pcs_4326 = 'EPSG:4326'  # WGS84

# 创建坐标转换器
transformer_to_4326 = Transformer.from_crs(pcs_3857, pcs_4326, always_xy=True)
transformer_to_3857 = Transformer.from_crs(pcs_4326, pcs_3857, always_xy=True)

# 读取湖泊边界
dir_path = r"C:\Users\Administrator\Desktop\test"
lake_boundary = gpd.read_file(os.path.join(dir_path, 'lakes_boundaries', '5lake_domain_merge.shp')).to_crs(pcs_3857)

# 创建输出目录
output_dir = os.path.join(dir_path, "lakes_image")
os.makedirs(output_dir, exist_ok=True)

def download_tile(tile, zoom, max_retries=5):
    """下载单个地图瓦片"""
    # url = f"https://mt{0-3}.google.com/vt/lyrs=s&x={x}&y={y}&z={zoom}"
    url = f"https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{zoom}/{tile.y}/{tile.x}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=15)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            if img.size == (256, 256):  # 验证图像尺寸
                return tile, img
            else:
                print(f"瓦片 {tile} 尺寸异常，尝试重试")
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"瓦片下载失败 {tile} 经过 {max_retries} 次尝试: {e}")
                return tile, None
            continue
    return tile, None

def calculate_pixel_bounds(transform, width, height):
    """计算图像的地理范围"""
    left = transform.c
    top = transform.f
    right = left + transform.a * width
    bottom = top + transform.e * height
    return left, bottom, right, top

def stitch_and_clip(tiles, tile_images, lake_geom_buffered, zoom, lake_name):
    """拼接瓦片并精确裁剪到buffered区域"""
    if not tiles or not tile_images:
        print(f"错误：无有效瓦片可处理 {lake_name}")
        return None

    # 计算拼接画布大小
    min_x = min(t.x for t in tiles)
    max_x = max(t.x for t in tiles)
    min_y = min(t.y for t in tiles)
    max_y = max(t.y for t in tiles)
    
    tile_size = 256
    width = (max_x - min_x + 1) * tile_size
    height = (max_y - min_y + 1) * tile_size
    
    # 创建空白画布
    stitched_img = Image.new('RGB', (width, height))
    
    # 拼接瓦片
    for tile, img in zip(tiles, tile_images):
        if img is None:
            continue
        x_offset = (tile.x - min_x) * tile_size
        y_offset = (max_y - tile.y) * tile_size  # 翻转Y轴以正确拼接
        stitched_img.paste(img, (x_offset, y_offset))
    
    # 计算地理范围
    def tile_to_bounds(t):
        return mercantile.bounds(t) if hasattr(mercantile, 'bounds') else mercantile.bounds(t.x, t.y, t.z)
    
    bounds_min = tile_to_bounds(mercantile.Tile(min_x, max_y, zoom))
    bounds_max = tile_to_bounds(mercantile.Tile(max_x, min_y, zoom))
    
    # 定义图像的地理范围（WGS84）
    transform = from_bounds(
        west=bounds_min.west,
        south=bounds_min.south,
        east=bounds_max.east,
        north=bounds_max.north,
        width=width,
        height=height
    )
    
    # 临时保存拼接后的图像
    temp_path = os.path.join(output_dir, f"temp_{lake_name}.tif")
    img_array = np.array(stitched_img)
    
    # 写入临时文件
    with rasterio.open(
        temp_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=3,
        dtype=img_array.dtype,
        crs=pcs_4326,
        transform=transform
    ) as dst:
        dst.write(img_array.transpose(2, 0, 1))
    
    # 精确裁剪
    try:
        with rasterio.open(temp_path) as src:
            # 确保湖泊buffered几何图形在WGS84
            lake_geom_buffered_4326 = gpd.GeoSeries([lake_geom_buffered], crs=pcs_3857).to_crs(pcs_4326).iloc[0]
            
            # 执行裁剪
            try:
                out_image, out_transform = mask(
                    src,
                    [lake_geom_buffered_4326],
                    crop=True,
                    all_touched=True,
                    pad=True
                )
            except ValueError as e:
                print(f"裁剪失败 {lake_name}: {e}")
                return None
            
            # 获取裁剪后的元数据
            out_meta = src.meta.copy()
            out_meta.update({
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
            
            # 保存裁剪结果
            clipped_path = os.path.join(output_dir, f"{lake_name}.tif")
            with rasterio.open(clipped_path, "w", **out_meta) as dest:
                dest.write(out_image)
            
            return clipped_path
    
    finally:
        # 删除临时文件
        if os.path.exists(temp_path):
            os.remove(temp_path)
def process_lake(row, idx, zoom=12):
    """处理单个湖泊"""
    lake_name = row.get('name', f'lake_{idx}')
    print(f"\n正在处理湖泊: {lake_name}")
    
    try:
        # 创建200米缓冲区并取包络矩形
        buffered = row.geometry.buffer(200)  # 200米缓冲
        
        # 获取包络矩形的范围（EPSG:3857）
        minx, miny, maxx, maxy = buffered.bounds
        # 转换为WGS84经纬度
        minx_lon, miny_lat = transformer_to_4326.transform(minx, miny)
        maxx_lon, maxy_lat = transformer_to_4326.transform(maxx, maxy)
        
        print(f"包络矩形经纬度范围: {minx_lon:.4f}, {miny_lat:.4f} -> {maxx_lon:.4f}, {maxy_lat:.4f}")
        
        # 计算所需瓦片
        try:
            tiles = list(mercantile.tiles(minx_lon, miny_lat, maxx_lon, maxy_lat, zooms=zoom))
        except TypeError:
            tiles = list(mercantile.tiles(minx_lon, miny_lat, maxx_lon, maxy_lat, zooms=zoom))
        
        if not tiles:
            print(f"警告：未获取到任何瓦片 {lake_name}")
            return False
        
        print(f"需要下载 {len(tiles)} 个瓦片")
        
        # 并行下载
        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            futures = [executor.submit(download_tile, tile, zoom) for tile in tiles]
            results = []
            for future in tqdm(concurrent.futures.as_completed(futures), 
                             total=len(tiles), desc="下载瓦片"):
                results.append(future.result())
        
        # 过滤掉下载失败的瓦片
        tiles_downloaded = []
        tile_images = []
        for tile, img in results:
            if img is not None:
                tiles_downloaded.append(tile)
                tile_images.append(img)
        
        if not tiles_downloaded:
            print(f"错误：所有瓦片下载失败 {lake_name}")
            return False
        
        # 拼接并裁剪到buffered区域
        output_path = stitch_and_clip(tiles_downloaded, tile_images, buffered, zoom, lake_name)
        
        if output_path:
            print(f"成功保存裁剪结果: {output_path}")
            return True
        else:
            print(f"错误：无法生成裁剪结果 {lake_name}")
            return False
    
    except Exception as e:
        print(f"处理湖泊 {lake_name} 时出错: {str(e)}")
        return False

# 主程序
if __name__ == "__main__":
    zoom_level = 14  # 推荐12-14级
    
    print(f"开始处理 {len(lake_boundary)} 个湖泊...")
    success_count = 0
    
    for idx, row in lake_boundary.iterrows():
        if process_lake(row, idx, zoom=zoom_level):
            success_count += 1
    
    print(f"\n处理完成！成功处理 {success_count}/{len(lake_boundary)} 个湖泊")
    print(f"裁剪后的结果保存在: {output_dir}")

# 绘图与交互式选点
保存绘图内容

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import rasterio
from pyproj import Transformer
import geopandas as gpd
import traceback
import seaborn as sns
import os

# 设置全局字体
plt.rcParams['font.family'] = 'Times New Roman'

# ======================
# 数据准备
# ======================
pcs_3857 = 'EPSG:3857'  # Web墨卡托
pcs_4326 = 'EPSG:4326'  # WGS84


shp_name = '5lake_domain_merge.shp'

dir_path = r"C:\Users\Administrator\Desktop\test"
folders = os.listdir(os.path.join(dir_path, 'ICESat-2_h5_files'))

for name in folders:
    # Construct paths inside the loop
    image_path = os.path.join(dir_path, 'lakes_image', f'{name}.tif')  # Fixed path construction
    DEM_path = os.path.join(dir_path, 'lakes_Bathymetry', f'LakeBathymetry_{name}_corrected.tif')
    domian_df = gpd.read_file(os.path.join(dir_path, 'lakes_boundaries', shp_name)).to_crs(pcs_3857)
    selected_area = domian_df[domian_df.name == name].geometry.area.iloc[0] / 1000 / 1000
    final_df = gpd.read_file(os.path.join(dir_path, 'filter_ATL03', f'{name}_filtered.csv'))

    final_df['lat_ph'] = final_df['lat_ph'].astype(float)
    final_df['lon_ph'] = final_df['lon_ph'].astype(float)
    final_df['h_ph']   = final_df['h_ph'].astype(float)

    unique_combinations = final_df[['cycle_number', 'rgt', 'beam']].drop_duplicates()
    print(f"共有 {len(unique_combinations)} 种唯一组合")

    # 遍历每种组合
    for idx, row in unique_combinations.iterrows():
        cycle, rgt, beam = row['cycle_number'], row['rgt'], row['beam']
        
        # 筛选当前组合的数据
        mask = (final_df['cycle_number'] == cycle) & \
                (final_df['rgt'] == rgt) & \
                (final_df['beam'] == beam)
        group_df = final_df[mask]

        # ======================
        # 创建图形
        # ======================
        fig = plt.figure(figsize=(14, 6))
        gs = GridSpec(1, 2, figure=fig, width_ratios=[1, 1])

        # 创建子图并设置相同高度
        ax1 = fig.add_subplot(gs[0])  # SAR图像
        ax2 = fig.add_subplot(gs[1])  # ICESat-2数据

        # 调整子图位置使高度一致
        pos1 = ax1.get_position()
        pos2 = ax2.get_position()
        ax2.set_position([pos2.x0, pos1.y0, pos2.width, pos1.height])  # 使右侧子图与左侧同高

        # ======================
        # 绘制Planet图像 (RGB显示)
        # ======================
        try:
            with rasterio.open(image_path) as src:
                # 读取所有波段
                bands = [src.read(i) for i in range(1, min(src.count, 4))]  # 读取前3个波段
                
                # 如果是单波段图像，转换为伪彩色
                if len(bands) == 1:
                    from matplotlib.colors import LinearSegmentedColormap
                    cmap = LinearSegmentedColormap.from_list('sar', ['black', 'white'])
                    img = ax1.imshow(bands[0], cmap=cmap, 
                                extent=[src.bounds.left, src.bounds.right, 
                                        src.bounds.bottom, src.bounds.top])
                else:
                    # 对多波段图像进行对比度拉伸
                    def stretch(band):
                        p2, p98 = np.percentile(band[band > 0], (2, 98))
                        return np.clip((band - p2) / (p98 - p2), 0, 1)
                    
                    rgb = np.dstack([stretch(band) for band in bands[:3]])
                    img = ax1.imshow(rgb, extent=[src.bounds.left, src.bounds.right, 
                                                src.bounds.bottom, src.bounds.top])
                
                # 绘制ICESat-2轨迹
                if not group_df.empty:
                    y_min, y_max = group_df['lat_ph'].min(), group_df['lat_ph'].max()
                    x_min = group_df.loc[group_df['lat_ph'].idxmin(), 'lon_ph']
                    x_max = group_df.loc[group_df['lat_ph'].idxmax(), 'lon_ph']
                            
                    # 绘制剖面线
                    ax1.plot([x_min, x_max], [y_min, y_max], 
                            color='red', linestyle='--', linewidth=2, 
                            label='ATL03 Line')
                    
                    ax1.scatter([x_min, x_max], [y_min, y_max], 
                            color='red', s=50, edgecolor='black')
                    
                ax1.text(
                        0.02, 0.98,
                        f"Selected Area: {selected_area:.5f} sq. km",
                        transform=ax1.transAxes,
                        fontsize=10,
                        color='black',
                        verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.5, edgecolor='none')
                    )
                
                ax1.set_title('SAR Image', fontsize=12)
                ax1.set_xlabel('Longitude', fontsize=10)
                ax1.set_ylabel('Latitude', fontsize=10)
                ax1.legend(loc='upper right', prop={'size': 10})

        except Exception as e:
            print(f"SAR图像加载错误: {e}")
            traceback.print_exc()
            ax1.set_title('SAR Image Not Available', fontsize=12)
            ax1.text(0.5, 0.5, str(e), ha='center', va='center')

        # ====================== 
        # 绘制ICESat-2数据
        # ======================
        if not group_df.empty:
            # 颜色映射
            tab10_colors = sns.color_palette("tab10")
            color_map = {
                'High': (tab10_colors[2], 'High'),
                'Median': (tab10_colors[0], 'Med'),
                'Low': (tab10_colors[4], 'Low'),
                'Buffer': (tab10_colors[1], 'Buffer'),
                'Noise': (tab10_colors[7], 'Noise')
            }
            
            # 绘制各分类点
            for class_type, (color, label) in color_map.items():
                subset = group_df[group_df['classification'] == class_type]
                if not subset.empty:
                    ax2.scatter(subset['lat_ph'], subset['h_ph'], 
                            color=color, s=8, alpha=0.3, label=label)
            
            # 修改后的DEM剖面提取代码
            try:
                with rasterio.open(DEM_path) as dem_src:
                    # 1. 获取ICESat-2轨迹的实际范围（UTM坐标）
                    transformer = Transformer.from_crs("EPSG:4326", dem_src.crs, always_xy=True)
                    
                    # 将ICESat-2轨迹端点转换到DEM的UTM坐标系
                    x_min_utm, y_min_utm = transformer.transform(x_min, y_min)
                    x_max_utm, y_max_utm = transformer.transform(x_max, y_max)
                    
                    print(f"\nICESat-2轨迹在DEM坐标系中的范围:")
                    print(f"X: [{min(x_min_utm, x_max_utm):.1f}, {max(x_min_utm, x_max_utm):.1f}]")
                    print(f"Y: [{min(y_min_utm, y_max_utm):.1f}, {max(y_min_utm, y_max_utm):.1f}]")
                    
                    # 2. 直接在UTM坐标系中采样（避免重投影损失）
                    num_points = 500  # 适当减少点数
                    x_coords_utm = np.linspace(x_min_utm, x_max_utm, num_points)
                    y_coords_utm = np.linspace(y_min_utm, y_max_utm, num_points)
                    
                    # 3. 获取高程值
                    dem_elevations = []
                    valid_indices = []
                    for i, (x, y) in enumerate(zip(x_coords_utm, y_coords_utm)):
                        if (dem_src.bounds.left <= x <= dem_src.bounds.right and 
                            dem_src.bounds.bottom <= y <= dem_src.bounds.top):
                            row, col = dem_src.index(x, y)
                            val = dem_src.read(1)[row, col]
                            if val != dem_src.nodata:
                                dem_elevations.append(val)
                                valid_indices.append(i)
                    
                    # 4. 转换有效点回WGS84用于绘图
                    transformer = Transformer.from_crs(dem_src.crs, "EPSG:4326", always_xy=True)
                    valid_x, valid_y = transformer.transform(
                        x_coords_utm[valid_indices], 
                        y_coords_utm[valid_indices]
                    )
                    
                    # 5. 计算沿轨距离（基于纬度）
                    along_track = np.sqrt((valid_x - valid_x[0])**2 + (valid_y - valid_y[0])**2) * 111.32  # 转换为km
                    
                    print(f"\n有效采样点: {len(dem_elevations)}/{num_points} ({(len(dem_elevations)/num_points)*100:.1f}%)")
                    
                    if len(dem_elevations) > 10:  # 至少有10个有效点才绘图
                        # 绘制DEM剖面线
                        ax2.plot(valid_y, dem_elevations, 
                                color='black', linewidth=2, linestyle='--',
                                label='DEM Profile')
                        
                        # 保存数据供检查
                        profile_data = np.column_stack([valid_y, dem_elevations, along_track])
                        np.savetxt(f"{name}_DEM_profile.csv", profile_data, delimiter=",", 
                                header="latitude,elevation,along_track_km", comments='')
                    else:
                        print("警告: 有效DEM采样点不足")
                        
            except Exception as e:
                print(f"DEM处理错误: {e}")
                traceback.print_exc()
            
            # 设置坐标范围和标签
            ax2.set_xlim(group_df['lat_ph'].min(), group_df['lat_ph'].max())
            ax2.set_ylim(group_df['h_ph'].min(), group_df['h_ph'].max())
            ax2.set_title('ICESat-2 Photon Data with DEM Profile', fontsize=12)
            ax2.set_xlabel('', fontsize=10)
            ax2.set_ylabel('Elevation (m)', fontsize=10)
            
            # 禁用科学计数法并设置4位小数
            ax2.ticklabel_format(axis='y', useOffset=False, style='plain')
            ax2.yaxis.set_major_formatter(plt.FormatStrFormatter('%.4f'))
            
            # 添加沿轨距离坐标轴
            transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857")
            x_min_3857, y_min_3857 = transformer.transform(y_min, x_min)
            x_max_3857, y_max_3857 = transformer.transform(y_max, x_max)
            
            distance_km = np.sqrt((x_max_3857 - x_min_3857)**2 + 
                                (y_max_3857 - y_min_3857)**2) / 1000
            
            def lat2dist(lat):
                return (lat - group_df['lat_ph'].min()) * distance_km / \
                    (group_df['lat_ph'].max() - group_df['lat_ph'].min())
            
            def dist2lat(dist):
                return group_df['lat_ph'].min() + \
                    dist * (group_df['lat_ph'].max() - group_df['lat_ph'].min()) / distance_km
            
            secax = ax2.secondary_xaxis(-0.075, functions=(lat2dist, dist2lat))
            secax.set_xlabel('Latitude/Along-Track Distance (km)', fontsize=10, labelpad=0)
            
            ax2.legend(loc='upper right', prop={'size': 8})
            
            # 计算众数，自适应绘制湖盆
            elevations = group_df['h_ph'][~np.isnan(group_df['h_ph'])]
            unique_values, counts = np.unique(elevations, return_counts=True)
            mode = unique_values[np.argmax(counts)]
            
            # 湖面level上下50m
            ax2.set_ylim(mode-100,mode + 100)
            
            
        else:
            ax2.set_title('No ICESat-2 Data Available', fontsize=12)

        plt.tight_layout()
        plt.show()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

        print(f"\nProcessing: Cycle {cycle}, RGT {rgt}, Beam {beam}")
        print(f"Number of points: {len(group_df)}")

# 交互式选点

In [24]:
cycle, rgt, beam

('24.0', '1102.0', 'gt1r')

In [33]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import rasterio
from pyproj import Transformer
import geopandas as gpd
import traceback
import seaborn as sns
import os
from matplotlib.patches import Polygon as MplPolygon
from shapely.geometry import LineString, Point
import pandas as pd

# 设置全局字体
plt.rcParams['font.family'] = 'Times New Roman'

# ======================
# 数据准备
# ======================
pcs_3857 = 'EPSG:3857'  # Web墨卡托
pcs_4326 = 'EPSG:4326'  # WGS84

shp_name = '5lake_domain_merge.shp'

dir_path = r"C:\Users\Administrator\Desktop\test"
folders = os.listdir(os.path.join(dir_path, 'ICESat-2_h5_files'))

name = 'ChamaqudanCo'

# Construct paths inside the loop
image_path = os.path.join(dir_path, 'lakes_image', f'{name}.tif')  # Fixed path construction
DEM_path = os.path.join(dir_path, 'lakes_Bathymetry', f'LakeBathymetry_{name}_corrected.tif')
domian_df = gpd.read_file(os.path.join(dir_path, 'lakes_boundaries', shp_name)).to_crs(pcs_3857)
selected_area = domian_df[domian_df.name == name].geometry.area.iloc[0] / 1000 / 1000
final_df = gpd.read_file(os.path.join(dir_path, 'filter_ATL03', f'{name}_filtered.csv'))

final_df['lat_ph'] = final_df['lat_ph'].astype(float)
final_df['lon_ph'] = final_df['lon_ph'].astype(float)
final_df['h_ph']   = final_df['h_ph'].astype(float)

unique_combinations = final_df[['cycle_number', 'rgt', 'beam']].drop_duplicates()
print(f"共有 {len(unique_combinations)} 种唯一组合")

# 遍历每种组合
cycle, rgt, beam = ('24.0', '1102.0', 'gt1r')
# 筛选当前组合的数据
mask = (final_df['cycle_number'] == cycle) & \
        (final_df['rgt'] == rgt) & \
        (final_df['beam'] == beam)
group_df = final_df[mask]

# ======================
# 创建图形
# ======================
fig = plt.figure(figsize=(14, 6))
gs = GridSpec(1, 2, figure=fig, width_ratios=[1, 1])

# 创建子图并设置相同高度
ax1 = fig.add_subplot(gs[0])  # SAR图像
ax2 = fig.add_subplot(gs[1])  # ICESat-2数据

# 调整子图位置使高度一致
pos1 = ax1.get_position()
pos2 = ax2.get_position()
ax2.set_position([pos2.x0, pos1.y0, pos2.width, pos1.height])  # 使右侧子图与左侧同高

# ======================
# 绘制Planet图像 (RGB显示)
# ======================
try:
    with rasterio.open(image_path) as src:
        # 读取所有波段
        bands = [src.read(i) for i in range(1, min(src.count, 4))]  # 读取前3个波段
        
        # 如果是单波段图像，转换为伪彩色
        if len(bands) == 1:
            from matplotlib.colors import LinearSegmentedColormap
            cmap = LinearSegmentedColormap.from_list('sar', ['black', 'white'])
            img = ax1.imshow(bands[0], cmap=cmap, 
                        extent=[src.bounds.left, src.bounds.right, 
                                src.bounds.bottom, src.bounds.top])
        else:
            # 对多波段图像进行对比度拉伸
            def stretch(band):
                p2, p98 = np.percentile(band[band > 0], (2, 98))
                return np.clip((band - p2) / (p98 - p2), 0, 1)
            
            rgb = np.dstack([stretch(band) for band in bands[:3]])
            img = ax1.imshow(rgb, extent=[src.bounds.left, src.bounds.right, 
                                        src.bounds.bottom, src.bounds.top])
        
        # 绘制ICESat-2轨迹
        if not group_df.empty:
            y_min, y_max = group_df['lat_ph'].min(), group_df['lat_ph'].max()
            x_min = group_df.loc[group_df['lat_ph'].idxmin(), 'lon_ph']
            x_max = group_df.loc[group_df['lat_ph'].idxmax(), 'lon_ph']
                    
            # 绘制剖面线
            ax1.plot([x_min, x_max], [y_min, y_max], 
                    color='red', linestyle='--', linewidth=2, 
                    label='ATL03 Line')
            
            ax1.scatter([x_min, x_max], [y_min, y_max], 
                    color='red', s=50, edgecolor='black')
            
        ax1.text(
                0.02, 0.98,
                f"Selected Area: {selected_area:.5f} sq. km",
                transform=ax1.transAxes,
                fontsize=10,
                color='black',
                verticalalignment='top',
                bbox=dict(facecolor='white', alpha=0.5, edgecolor='none')
            )
        
        ax1.set_title('SAR Image', fontsize=12)
        ax1.set_xlabel('Longitude', fontsize=10)
        ax1.set_ylabel('Latitude', fontsize=10)
        ax1.legend(loc='upper right', prop={'size': 10})

except Exception as e:
    print(f"SAR图像加载错误: {e}")
    traceback.print_exc()
    ax1.set_title('SAR Image Not Available', fontsize=12)
    ax1.text(0.5, 0.5, str(e), ha='center', va='center')

# ====================== 
# 绘制ICESat-2数据
# ======================
if not group_df.empty:
    # 颜色映射
    tab10_colors = sns.color_palette("tab10")
    color_map = {
        'High': (tab10_colors[2], 'High'),
        'Median': (tab10_colors[0], 'Med'),
        'Low': (tab10_colors[4], 'Low'),
        'Buffer': (tab10_colors[1], 'Buffer'),
        'Noise': (tab10_colors[7], 'Noise')
    }
    
    # 绘制各分类点
    for class_type, (color, label) in color_map.items():
        subset = group_df[group_df['classification'] == class_type]
        if not subset.empty:
            ax2.scatter(subset['lat_ph'], subset['h_ph'], 
                    color=color, s=8, alpha=0.3, label=label)
    
    # 修改后的DEM剖面提取代码
    try:
        with rasterio.open(DEM_path) as dem_src:
            # 1. 获取ICESat-2轨迹的实际范围（UTM坐标）
            transformer = Transformer.from_crs("EPSG:4326", dem_src.crs, always_xy=True)
            
            # 将ICESat-2轨迹端点转换到DEM的UTM坐标系
            x_min_utm, y_min_utm = transformer.transform(x_min, y_min)
            x_max_utm, y_max_utm = transformer.transform(x_max, y_max)
            
            print(f"\nICESat-2轨迹在DEM坐标系中的范围:")
            print(f"X: [{min(x_min_utm, x_max_utm):.1f}, {max(x_min_utm, x_max_utm):.1f}]")
            print(f"Y: [{min(y_min_utm, y_max_utm):.1f}, {max(y_min_utm, y_max_utm):.1f}]")
            
            # 2. 直接在UTM坐标系中采样（避免重投影损失）
            num_points = 500  # 适当减少点数
            x_coords_utm = np.linspace(x_min_utm, x_max_utm, num_points)
            y_coords_utm = np.linspace(y_min_utm, y_max_utm, num_points)
            
            # 3. 获取高程值
            dem_elevations = []
            valid_indices = []
            for i, (x, y) in enumerate(zip(x_coords_utm, y_coords_utm)):
                if (dem_src.bounds.left <= x <= dem_src.bounds.right and 
                    dem_src.bounds.bottom <= y <= dem_src.bounds.top):
                    row, col = dem_src.index(x, y)
                    val = dem_src.read(1)[row, col]
                    if val != dem_src.nodata:
                        dem_elevations.append(val)
                        valid_indices.append(i)
            
            # 4. 转换有效点回WGS84用于绘图
            transformer = Transformer.from_crs(dem_src.crs, "EPSG:4326", always_xy=True)
            valid_x, valid_y = transformer.transform(
                x_coords_utm[valid_indices], 
                y_coords_utm[valid_indices]
            )
            
            # 5. 计算沿轨距离（基于纬度）
            along_track = np.sqrt((valid_x - valid_x[0])**2 + (valid_y - valid_y[0])**2) * 111.32  # 转换为km
            
            print(f"\n有效采样点: {len(dem_elevations)}/{num_points} ({(len(dem_elevations)/num_points)*100:.1f}%)")
            
            if len(dem_elevations) > 10:  # 至少有10个有效点才绘图
                # 绘制DEM剖面线
                ax2.plot(valid_y, dem_elevations, 
                        color='black', linewidth=2, linestyle='--',
                        label='DEM Profile')
                
                # 保存数据供检查
                profile_data = np.column_stack([valid_y, dem_elevations, along_track])
                np.savetxt(f"{name}_DEM_profile.csv", profile_data, delimiter=",", 
                        header="latitude,elevation,along_track_km", comments='')
            else:
                print("警告: 有效DEM采样点不足")
                
    except Exception as e:
        print(f"DEM处理错误: {e}")
        traceback.print_exc()
    
    # 设置坐标范围和标签
    ax2.set_xlim(group_df['lat_ph'].min(), group_df['lat_ph'].max())
    ax2.set_ylim(group_df['h_ph'].min(), group_df['h_ph'].max())
    ax2.set_title('ICESat-2 Photon Data with DEM Profile', fontsize=12)
    ax2.set_xlabel('', fontsize=10)
    ax2.set_ylabel('Elevation (m)', fontsize=10)
    
    # 禁用科学计数法并设置4位小数
    ax2.ticklabel_format(axis='y', useOffset=False, style='plain')
    ax2.yaxis.set_major_formatter(plt.FormatStrFormatter('%.4f'))
    
    # 添加沿轨距离坐标轴
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    x_min_3857, y_min_3857 = transformer.transform(y_min, x_min)
    x_max_3857, y_max_3857 = transformer.transform(y_max, x_max)
    
    distance_km = np.sqrt((x_max_3857 - x_min_3857)**2 + 
                        (y_max_3857 - y_min_3857)**2) / 1000
    
    def lat2dist(lat):
        return (lat - group_df['lat_ph'].min()) * distance_km / \
            (group_df['lat_ph'].max() - group_df['lat_ph'].min())
    
    def dist2lat(dist):
        return group_df['lat_ph'].min() + \
            dist * (group_df['lat_ph'].max() - group_df['lat_ph'].min()) / distance_km
    
    secax = ax2.secondary_xaxis(-0.075, functions=(lat2dist, dist2lat))
    secax.set_xlabel('Latitude/Along-Track Distance (km)', fontsize=10, labelpad=0)
    
    ax2.legend(loc='upper right', prop={'size': 8})
    
    # 计算众数，自适应绘制湖盆
    elevations = group_df['h_ph'][~np.isnan(group_df['h_ph'])]
    unique_values, counts = np.unique(elevations, return_counts=True)
    mode = unique_values[np.argmax(counts)]
    
    # 湖面level上下50m
    ax2.set_ylim(mode-100,mode + 100)
    
    # ===========================================
    # 添加点选功能
    # ===========================================
    print("\nInteractive point selection mode activated")
    print("Click on the right plot to select points (press Enter to finish)")
    
    # 启用交互模式
    plt.ion()
    plt.show()
    
    # 收集用户选择的点
    selected_points = plt.ginput(n=-1, timeout=0)
    
    if selected_points:
        # 将点转换为DataFrame
        selected_lats = [p[0] for p in selected_points]
        selected_heights = [p[1] for p in selected_points]
        
        # 找到每个纬度点对应的经度
        selected_lons = []
        for lat in selected_lats:
            # 找到最接近的纬度点
            closest_idx = np.abs(group_df['lat_ph'] - lat).idxmin()
            selected_lons.append(group_df.loc[closest_idx, 'lon_ph'])
        
        # 创建包含所有信息的DataFrame
        selection_df = pd.DataFrame({
            'longitude': selected_lons,
            'latitude': selected_lats,
            'elevation': selected_heights
        })
        
        # 在图上绘制选中的点
        ax2.scatter(selected_lats, selected_heights, 
                   color='red', s=50, edgecolor='black', 
                   label='Selected Points')
        
        # 连接选中的点形成线
        sorted_points = sorted(zip(selected_lats, selected_heights), key=lambda x: x[0])
        sorted_lats = [p[0] for p in sorted_points]
        sorted_heights = [p[1] for p in sorted_points]
        ax2.plot(sorted_lats, sorted_heights, 
                color='red', linewidth=1.5, linestyle='-',
                label='Selected Line')
        
        ax2.legend(loc='upper right', prop={'size': 8})
        
        # 创建GeoDataFrame保存结果
        geometry = [Point(lon, lat) for lon, lat in zip(selected_lons, selected_lats)]
        gdf = gpd.GeoDataFrame(selection_df, geometry=geometry, crs="EPSG:4326")
        
        # 保存为GeoJSON
        output_path = os.path.join(dir_path, f'{name}_selected_points.geojson')
        gdf.to_file(output_path, driver='GeoJSON')
        print(f"\nSelected points saved to: {output_path}")
    
    # 禁用交互模式
    plt.ioff()
    
else:
    ax2.set_title('No ICESat-2 Data Available', fontsize=12)

plt.tight_layout()
plt.show()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

print(f"\nProcessing: Cycle {cycle}, RGT {rgt}, Beam {beam}")
print(f"Number of points: {len(group_df)}")

共有 10 种唯一组合

Interactive point selection mode activated
Click on the right plot to select points (press Enter to finish)


KeyboardInterrupt: 

# 三维湖盆重构